# Getting started with MovingPandas

<img align="right" src="https://anitagraser.github.io/movingpandas/assets/img/movingpandas.png">

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/anitagraser/movingpandas/master?filepath=tutorials/1-getting-started.ipynb)

**<p style="color:#e31883">This notebook demonstrates the current development version of MovingPandas.</p>**

For tutorials using the latest release visit https://github.com/anitagraser/movingpandas-examples.


In [ ]:
import urllib
import os
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from fiona.crs import from_epsg
from datetime import datetime, timedelta
from matplotlib import pyplot as plt

import sys
sys.path.append("..")
import movingpandas as mpd
mpd.show_versions()

import warnings
#warnings.simplefilter("ignore")

## Creating a trajectory from scratch


In [ ]:
df = pd.DataFrame([
  {'geometry':Point(0,0), 't':datetime(2018,1,1,12,0,0)},
  {'geometry':Point(6,0), 't':datetime(2018,1,1,12,6,0)},
  {'geometry':Point(6,6), 't':datetime(2018,1,1,12,10,0)},
  {'geometry':Point(9,9), 't':datetime(2018,1,1,12,15,0)}
]).set_index('t')
geo_df = GeoDataFrame(df, crs=31256)
toy_traj = mpd.Trajectory(geo_df, 1)
toy_traj.df

In [ ]:
toy_traj.to_line_gdf()

In [ ]:
toy_traj.to_traj_gdf(wkt=True)

We can access **key information** about our trajectory by looking at the print output:

In [ ]:
print(toy_traj)

We can also access the trajectories GeoDataFrame:

In [ ]:
toy_traj.df

## Visualizing trajectories

To **visualize the trajectory**, we can turn it into a linestring.

(The notebook environment automatically plots Shapely geometry objects like the LineString returned by to_linestring().)

In [ ]:
toy_traj.to_linestring()

We can **compute the speed** of movement along the trajectory (between consecutive points). The values are in meters per second:

In [ ]:
toy_traj.add_speed(overwrite=True)
toy_traj.df

We can also visualize the speed values:

In [ ]:
toy_traj.plot(column="speed", linewidth=5, capstyle='round', legend=True)

In contrast to the earlier example where we visualized the whole trajectory as one linestring, the trajectory plot() function draws each line segment individually and thus each can have a different color.

## Analyzing trajectories

### Extracting a moving object's position was at a certain time

For example, let's have a look at the get_position_at() function:

In [ ]:
toy_traj.get_position_at(datetime(2018,1,1,12,6,0), method="nearest")    

To see its coordinates, we can look at the print output:

In [ ]:
print(toy_traj.get_position_at(datetime(2018,1,1,12,6,0), method="nearest"))

The method parameter describes what the function should do if there is no entry in the trajectory GeoDataFrame for the specified timestamp. 

For example, there is no entry at 2018-01-01 12:07:00

In [ ]:
toy_traj.df

In [ ]:
print(toy_traj.get_position_at(datetime(2018,1,1,12,7,0), method="nearest"))
print(toy_traj.get_position_at(datetime(2018,1,1,12,7,0), method="interpolated"))
print(toy_traj.get_position_at(datetime(2018,1,1,12,7,0), method="ffill")) # from the previous row
print(toy_traj.get_position_at(datetime(2018,1,1,12,7,0), method="bfill")) # from the following row

### Extracting trajectory segments based on time or geometry (i.e. clipping)

First, let's extract the trajectory segment for a certain time period:

In [ ]:
segment = toy_traj.get_segment_between(datetime(2018,1,1,12,6,0),datetime(2018,1,1,12,12,0))
print(segment)

Now, let's extract the trajectory segment that intersects with a given polygon:

In [ ]:
xmin, xmax, ymin, ymax = 2, 8, -10, 5
polygon = Polygon([(xmin, ymin), (xmin, ymax), (xmax, ymax), (xmax, ymin), (xmin, ymin)])
polygon

In [ ]:
intersections = toy_traj.clip(polygon)
intersections

In [ ]:
intersections.trajectories[0].plot(linewidth=5, capstyle='round')

## Beyond toy trajectories: Loading trajectory data from GeoPackage

The MovingPandas repository contains a demo GeoPackage file that can be loaded as follows:

In [ ]:
%%time
gdf = read_file('data/demodata_geolife.gpkg')
print("Finished reading {} rows".format(len(df)))

After reading the trajectory point data from file, we want to construct the trajectories.

### Creating trajectories with TrajectoryCollection

TrajectoryCollection is a convenience class that takes care of creating trajectories from a GeoDataFrame:

In [ ]:
traj_collection = mpd.TrajectoryCollection(gdf, 'trajectory_id', t='t')
print(traj_collection)

In [ ]:
traj_collection.plot(column='trajectory_id', legend=True, figsize=(9,5))

### Converting TrajectoryCollections back to GeoDataFrames



In [ ]:
traj_collection.to_point_gdf()

In [ ]:
traj_collection.to_line_gdf()

In [ ]:
traj_collection.to_traj_gdf(wkt=True)

These GeoDataFrames can be exported to different file formats using GeoPandas, as documented in https://geopandas.org/docs/user_guide/io.html

In [ ]:
traj_collection.to_traj_gdf(wkt=True).to_file("temp.gpkg", layer='trajectories', driver="GPKG")

In [ ]:
read_file('temp.gpkg').plot()

### Visualizing real-world trajectories

In [ ]:
my_traj = traj_collection.trajectories[1]
print(my_traj)

In [ ]:
plot_defaults = {'linewidth':5, 'capstyle':'round', 'figsize':(9,3), 'legend':True}
my_traj.plot(column='speed', vmax=20, **plot_defaults)

To visualize trajectories in their geographical context, we can also create interactive plots with basemaps:

In [ ]:
hvplot_defaults = {'tiles':'CartoLight', 'frame_height':400, 'frame_width':700, 'cmap':'Viridis', 'colorbar':True}
my_hvplot = my_traj.hvplot(c='speed', line_width=7.0, clim=(0,20), **hvplot_defaults)
my_hvplot

And even put other layers on top:

In [ ]:
my_hvplot * gpd.GeoDataFrame([my_traj.get_row_at(datetime(2009,6,29,8,0,0))]).hvplot(geo=True, size=200, color='red')

## Trajectory manipulation and handling

### Finding intersections with a Shapely polygon

The clip function can be used to extract trajectory segments that are located within an area of interest polygon.

This is how to use clip on a list of Trajectory objects:

In [ ]:
xmin, xmax, ymin, ymax = 116.3685035,116.3702945,39.904675,39.907728
polygon = Polygon([(xmin,ymin), (xmin,ymax), (xmax,ymax), (xmax,ymin), (xmin,ymin)])

intersections = []
for traj in traj_collection:
    for intersection in traj.clip(polygon):
        intersections.append(intersection)
print("Found {} intersections".format(len(intersections)))

In [ ]:
intersections[2].plot(linewidth=5.0, capstyle='round')

Alternatively, using **TrajectoryCollection**:

In [ ]:
clipped = traj_collection.clip(polygon)
clipped.trajectories[2].plot(linewidth=5.0, capstyle='round')

### Splitting trajectories

Gaps are quite common in trajectories. For example, GPS tracks may contain gaps if moving objects enter tunnels where GPS reception is lost. In other use cases, moving objects may leave the observation area for longer time before returning and continuing their recorded track.

Depending on the use case, we therefore might want to split trajectories at observation gaps that exceed a certain minimum duration:

In [ ]:
my_traj = traj_collection.trajectories[1]
print(my_traj)
my_traj.plot(column='speed', vmax=20, **plot_defaults)

In [ ]:
split = mpd.ObservationGapSplitter(my_traj).split(gap=timedelta(minutes=5))
for traj in split:
    print(traj)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=len(split), figsize=(19,4))
for i, traj in enumerate(split):
    traj.plot(ax=axes[i], column='speed', vmax=20, **plot_defaults)

In [ ]:
split = mpd.StopSplitter(my_traj).split(min_duration=timedelta(minutes=1), max_diameter=30, min_length=500)
for traj in split:
    print(traj)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=len(split), figsize=(19,4))
for i, traj in enumerate(split):
    traj.plot(ax=axes[i], column='speed', vmax=20, **plot_defaults)

Alternatively, using the whole **TrajectoryCollection**:

In [ ]:
split = mpd.ObservationGapSplitter(traj_collection).split(gap=timedelta(minutes=15))
print(split)
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16,4))
traj_collection.plot(ax=axes[0], column='trajectory_id', **plot_defaults)
axes[0].set_title('Original Trajectories')
split.plot(ax=axes[1], column='trajectory_id', **plot_defaults)
axes[1].set_title('Split Trajectories')

### Generalizing trajectories

To reduce the size of trajectory objects, we can generalize them, for example, using the Douglas-Peucker algorithm:

In [ ]:
original_traj = traj_collection.trajectories[1]
print(original_traj)

In [ ]:
original_traj.plot(column='speed', vmax=20, **plot_defaults)

Try different tolerance settings and observe the results in line geometry and therefore also length:

#### Spatial generalizers

In [ ]:
dp_generalized = mpd.DouglasPeuckerGeneralizer(original_traj).generalize(tolerance=0.001)
dp_generalized.plot(column='speed', vmax=20, **plot_defaults)

In [ ]:
print('Original length: %s'%(original_traj.get_length()))
print('Generalized length: %s'%(dp_generalized.get_length()))

An alternative generalization method is to down-sample the trajectory to ensure a certain time delta between records:

#### Temporal generalizers

In [ ]:
time_generalized = mpd.MinTimeDeltaGeneralizer(original_traj).generalize(tolerance=timedelta(minutes=1))
time_generalized.plot(column='speed', vmax=20, **plot_defaults)

In [ ]:
time_generalized.to_point_gdf().head(10)

In [ ]:
original_traj.to_point_gdf().head(10)

#### Spatiotemporal generalizers

In [ ]:
tdtr_generalized = mpd.TopDownTimeRatioGeneralizer(original_traj).generalize(tolerance=0.001)

Let's compare this to the basic Douglas-Peucker result:

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(19,4))
tdtr_generalized.plot(ax=axes[0], column='speed', vmax=20, **plot_defaults)
dp_generalized.plot(ax=axes[1], column='speed', vmax=20, **plot_defaults)

### Smoothing trajectories

We use the `KalmanSmootherCV` trajectory smoother to smooth all trajectories based on the assumption of a **nearly-constant velocity (CV) model**. The `process_noise_std` and `measurement_noise_std` parameters can be used to tune the smoother:

- `process_noise_std` governs the uncertainty associated with the adherence of the new (smooth) trajectories to the CV model assumption; higher values relax the assumption, therefore leading to less-smooth trajectories, and vice-versa. 
- `measurement_noise_std` controls the assumed error in the original trajectories; higher values dictate that the original trajectories are expected to be noisier (and therefore, less reliable), thus leading to smoother trajectories, and vice-versa. 

Try tuning these parameters and observe the resulting trajectories:

In [ ]:
smooth = mpd.trajectory_smoother.KalmanSmootherCV(split).smooth(process_noise_std=0.1, measurement_noise_std=10)
print(smooth)

Let's visually compare the smooth and original (split) trajectories.

In [ ]:
hvplot_defaults = {'tiles':'CartoLight', 'frame_height':320, 'frame_width':320, 'cmap':'Viridis', 'colorbar':True}
kwargs = {**hvplot_defaults, 'line_width':4}
(split.hvplot(title='Original Trajectories', **kwargs) + 
 smooth.hvplot(title='Smooth Trajectories', **kwargs))

And finally, let's compare the calculated speeds:

In [ ]:
kwargs = {**hvplot_defaults, 'c':'speed', 'line_width':7, 'clim':(0,20)}
(split.trajectories[2].hvplot(title='Original Trajectories', **kwargs) + 
 smooth.trajectories[2].hvplot(title='Smooth Trajectories', **kwargs))

As expected, the smooth trajectories are significantly less rugged (i.e. noisy) and exhibit smoother velocity transitions.

## Continue exploring MovingPandas

1. [Getting started](1-getting-started.ipynb)
1. [Handling trajectory data files (reading & writing)](2-reading-data-from-files.ipynb)
1. [TrajectoryCollection aggregation (flow maps)](3-generalization-and-aggregation.ipynb)
1. [Stop detection](4-stop-detection.ipynb)
1. [Working with local coordinates](5-local-coordinates.ipynb)